In [1]:
!pip install polars
!pip install glob2
!pip install boto3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import polars as pl
import numpy as np
import glob
import boto3
from io import BytesIO
from botocore.exceptions import NoCredentialsError

In [3]:
def concatenate_files_to_dataframe(bucket_name, prefix,sep,how):
    try:
        # Listar os objetos no bucket com o prefixo especificado
        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

        # Inicializar uma lista para armazenar os DataFrames de cada arquivo
        dataframes = []

        # Processar cada objeto listado
        for obj in objects.get('Contents', []):
            key = obj['Key']

            # Baixar o objeto como um stream de bytes
            response = s3.get_object(Bucket=bucket_name, Key=key)
            data = response['Body'].read()
            
            # Decodificar o stream de bytes usando a codificação correta
            #decoded_data = data.decode('iso-8859-1')

            # Processar o stream de bytes como um DataFrame (usando pandas neste exemplo)
            try:
                dataframe = pl.read_csv(BytesIO(data),separator=sep,encoding='utf8-lossy')
            except:
                next
            # Adicionar o DataFrame à lista
            dataframes.append(dataframe)

        # Concatenar todos os DataFrames em um único DataFrame
        concatenated_dataframe = pl.concat(dataframes, how=how)

        # Mostrar as primeiras linhas do DataFrame concatenado
        print(concatenated_dataframe.head())

    except NoCredentialsError:
        print("Credenciais não encontradas. Certifique-se de configurar suas credenciais AWS.")
    return concatenated_dataframe

In [4]:
def save_dataframe_to_s3(dataframe, destination_bucket, destination_key):
    try:
        # Salvar o DataFrame como um arquivo Parquet em um objeto BytesIO
        buffer = BytesIO()
        dataframe.write_parquet(buffer)

        # Fazer o upload do arquivo Parquet para o novo bucket S3
        s3.put_object(Bucket=destination_bucket, Key=destination_key, Body=buffer.getvalue())

        print("DataFrame tratado salvo com sucesso no novo bucket S3!")

    except NoCredentialsError:
        print("Credenciais não encontradas. Certifique-se de configurar suas credenciais AWS.")



# empregados

In [5]:
# Configurações
bucket_name = 'raw-data-638026687251'
prefix = 'Empregados/'
# Inicializar o cliente S3
s3 = boto3.client('s3')
# Chamar a função para iniciar o download e processamento
df_empregados = concatenate_files_to_dataframe(bucket_name, prefix,'|','diagonal')
df_empregados = df_empregados[['Nome','Geral','Remuneração e benefícios']]
df_empregados.head(1)

shape: (5, 24)
┌────────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬──────────┐
│ employer_n ┆ reviews_c ┆ culture_c ┆ salaries_ ┆ … ┆ CNPJ     ┆ Nome      ┆ match_per ┆ Segmento │
│ ame        ┆ ount      ┆ ount      ┆ count     ┆   ┆ ---      ┆ ---       ┆ cent      ┆ ---      │
│ ---        ┆ ---       ┆ ---       ┆ ---       ┆   ┆ i64      ┆ str       ┆ ---       ┆ str      │
│ str        ┆ i64       ┆ i64       ┆ i64       ┆   ┆          ┆           ┆ i64       ┆          │
╞════════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪══════════╡
│ Apex Group ┆ 450       ┆ 281       ┆ 592       ┆ … ┆ 655522   ┆ APE       ┆ 70        ┆ null     │
│            ┆           ┆           ┆           ┆   ┆          ┆ POUPEX    ┆           ┆          │
│ Banco Alfa ┆ 175       ┆ 74        ┆ 271       ┆ … ┆ 58160789 ┆ SAFRA     ┆ 67        ┆ null     │
│ China Cons ┆ 871       ┆ 194       ┆ 720       ┆ … ┆ 7450604  ┆ CHINA CON 

Nome,Geral,Remuneração e benefícios
str,f64,f64
"""APE POUPEX""",3.2,2.7


# bancos

In [6]:
# Configurações
bucket_name = 'raw-data-638026687251'
prefix = 'Bancos/'
# Inicializar o cliente S3
s3 = boto3.client('s3')
# Chamar a função para iniciar o download e processamento
df_bancos = concatenate_files_to_dataframe(bucket_name, prefix,'\t','diagonal')

df_bancos = df_bancos.with_columns(pl.col('Nome').str.split_exact(" - ", 1).struct.rename_fields(["Nome do Banco", "second_part"]).alias("fields")).unnest("fields")
df_bancos = df_bancos.drop(columns=['Nome','second_part'])
df_bancos.head(1)

shape: (5, 3)
┌──────────┬──────────┬───────────────────────────────────┐
│ Segmento ┆ CNPJ     ┆ Nome                              │
│ ---      ┆ ---      ┆ ---                               │
│ str      ┆ i64      ┆ str                               │
╞══════════╪══════════╪═══════════════════════════════════╡
│ S1       ┆ 0        ┆ BANCO DO BRASIL - PRUDENCIAL      │
│ S1       ┆ 60746948 ┆ BRADESCO - PRUDENCIAL             │
│ S1       ┆ 30306294 ┆ BTG PACTUAL - PRUDENCIAL          │
│ S1       ┆ 360305   ┆ CAIXA ECONOMICA FEDERAL - PRUDEN… │
│ S1       ┆ 60872504 ┆ ITAU - PRUDENCIAL                 │
└──────────┴──────────┴───────────────────────────────────┘


Segmento,CNPJ,Nome do Banco
str,i64,str
"""S1""",0,"""BANCO DO BRASI…"


# reclamações

In [7]:
# Configurações
bucket_name = 'raw-data-638026687251'
prefix = 'Reclamações/'
# Inicializar o cliente S3
s3 = boto3.client('s3')
# Chamar a função para iniciar o download e processamento
df_reclamacoes = concatenate_files_to_dataframe(bucket_name, prefix,';','vertical_relaxed')

df_reclamacoes = df_reclamacoes.with_columns(pl.col('Institui��o financeira').str.split_exact(" (", 1).struct.rename_fields(["Nome do Banco", "second_part"]).alias("fields")).unnest("fields")
df_reclamacoes = df_reclamacoes[["Nome do Banco", '�ndice','Quantidade total de reclama��es', 'Quantidade total de clientes � CCS e SCR']]
df_reclamacoes = df_reclamacoes.with_columns(pl.col('�ndice').str.replace('.',''))
df_reclamacoes = df_reclamacoes.with_columns(pl.col('�ndice').str.replace(',','.'))
df_reclamacoes.head(1)

shape: (5, 14)
┌──────┬───────────┬────────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ Ano  ┆ Trimestre ┆ Categoria  ┆ Tipo       ┆ … ┆ Quantidade ┆ Quantidade ┆ Quantidad ┆ Quantidad │
│ ---  ┆ ---       ┆ ---        ┆ ---        ┆   ┆ total de   ┆ total de   ┆ e de      ┆ e de      │
│ i64  ┆ str       ┆ str        ┆ str        ┆   ┆ reclama��e ┆ clientes � ┆ clientes  ┆ clientes  │
│      ┆           ┆            ┆            ┆   ┆ s          ┆ C…         ┆ � CCS     ┆ � SCR     │
│      ┆           ┆            ┆            ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│      ┆           ┆            ┆            ┆   ┆ i64        ┆ str        ┆ str       ┆ str       │
╞══════╪═══════════╪════════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2021 ┆ 1�        ┆ Grupo      ┆ Conglomera ┆ … ┆ 9          ┆ 26230      ┆ 24698     ┆ 3810      │
│      ┆           ┆ Secund�rio ┆ do         ┆   ┆            ┆            ┆

Nome do Banco,�ndice,Quantidade total de reclama��es,Quantidade total de clientes � CCS e SCR
str,str,i64,str
"""ABC-BRASIL""","""""",9,"""26230"""


In [8]:
df_join1=df_bancos.join(df_empregados,left_on='Nome do Banco',right_on='Nome',how='inner')

In [9]:
df_join1.head(1)

Segmento,CNPJ,Nome do Banco,Geral,Remuneração e benefícios
str,i64,str,f64,f64
"""S1""",0,"""BANCO DO BRASI…",4.3,4.2


In [10]:
df_join2 = df_bancos.join(df_reclamacoes,on='Nome do Banco',how='inner')

In [11]:
df_join2.head(1)

Segmento,CNPJ,Nome do Banco,�ndice,Quantidade total de reclama��es,Quantidade total de clientes � CCS e SCR
str,i64,str,str,i64,str
"""S1""",60746948,"""BRADESCO""","""2.78""",8752,"""98648524"""


In [12]:
df_join2 = df_join2.with_columns(pl.col(['�ndice']).cast(pl.Float64,strict=False))
df_join2 = df_join2.with_columns(pl.col(['Quantidade total de reclama��es', 'Quantidade total de clientes � CCS e SCR']).cast(pl.Int64))

In [13]:
df_join2 = df_join2.groupby(['CNPJ','Nome do Banco']).agg(
    pl.col('�ndice').mean(),
    pl.col('Quantidade total de reclama��es').sum(),
    pl.col('Quantidade total de clientes � CCS e SCR').sum()
    )

In [14]:
df_final = df_join1.join(df_join2,on='CNPJ',how='inner')

In [15]:
df_final.head(1)

Segmento,CNPJ,Nome do Banco,Geral,Remuneração e benefícios,Nome do Banco_right,�ndice,Quantidade total de reclama��es,Quantidade total de clientes � CCS e SCR
str,i64,str,f64,f64,str,f64,i64,i64
"""S1""",60872504,"""ITAU""",4.5,4.6,"""ITAU""",4.9125,48702,725005836


In [16]:
df_final = df_final.rename({'Segmento':'Classificação do Banco',
                 'Quantidade total de clientes � CCS e SCR':'Quantidade de Clientes do Bancos',
                 '�ndice':'Índice de reclamações',
                 'Quantidade total de reclama��es':'Quantidade de reclamações',
                 'Geral':'Índice de satisfação dos funcionários dos bancos',
                 'Remuneração e benefícios':'Índice de satisfação com salários dos funcionários dos bancos'})
df_final = df_final.drop(columns=['Nome do Banco_right'])
df_final = df_final[['Nome do Banco',
'CNPJ',
'Classificação do Banco',
'Quantidade de Clientes do Bancos',
'Índice de reclamações',
'Quantidade de reclamações',
'Índice de satisfação dos funcionários dos bancos',
'Índice de satisfação com salários dos funcionários dos bancos']]

In [17]:
df_final

Nome do Banco,CNPJ,Classificação do Banco,Quantidade de Clientes do Bancos,Índice de reclamações,Quantidade de reclamações,Índice de satisfação dos funcionários dos bancos,Índice de satisfação com salários dos funcionários dos bancos
str,i64,str,i64,f64,i64,f64,f64
"""ITAU""",60872504,"""S1""",725005836,4.9125,48702,4.5,4.6
"""BANRISUL""",92702067,"""S2""",40842351,4.10125,4092,4.2,4.1
"""CREDIT SUISSE""",33987793,"""S2""",5997,null,4,3.8,3.6
"""JP MORGAN CHAS…",33172537,"""S3""",15661,null,12,4.0,3.9
"""JP MORGAN CHAS…",33172537,"""S3""",15661,null,12,4.1,3.9
"""ORIGINAL""",92894922,"""S3""",216530140,5.80875,12093,3.8,4.0
"""SICREDI""",1181521,"""S3""",45197272,null,19,4.4,4.1
"""CITIBANK""",33479023,"""S2""",145999,null,105,3.9,3.8
"""ALFA""",60770336,"""S3""",3385769,null,537,3.5,3.8


In [18]:
destination_bucket_name = 'process-data-638026687251'
destination_key = 'df_final.parquet'

# Chamar a função para salvar o DataFrame tratado no novo bucket S3
save_dataframe_to_s3(df_final, destination_bucket_name, destination_key)

DataFrame tratado salvo com sucesso no novo bucket S3!
